In [33]:
import os
import io
import pandas as pd
import sys
import numpy as np
import twitter
from twitter import error
import json
import time
import random

import google.cloud
from google.cloud import datastore
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

from sqlalchemy import *
import sqlalchemy
import sqlalchemy.schema
from sqlalchemy.types import *

from IPython.display import *
from pandas.io.json import *
# gcloud auth application-default login

In [34]:
#client = datastore.Client()
#client = language.LanguageServiceClient()

In [35]:
api = twitter.Api(consumer_key="NOrVFOrKaGpOsD1LHjlDmD9lA",
                  consumer_secret="8Lg3hdsmIbEn9Q1fLTiWbCqHD7vp2OSc21mvTGElnudCr5jM7p",
                  access_token_key="49683260-3LQ6XoNAFi9R7MVaSGU1igwNxJkfxL6aFpUJfe7pG",
                  access_token_secret="zNXNyrewzVYC2kIqQ1GlGQoLIDHmTJtjXpMFZhwKGxYb8")

In [36]:
#https://dev.twitter.com/rest/reference/get/search/tweets
# https://python-twitter.readthedocs.io/en/latest/searching.html
# https://www.karambelkar.info/2015/01/how-to-use-twitters-search-rest-api-most-effectively
sinceId = None
maxTweets = 10000000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
max_id = -1
tweetCount = 0
results = api.GetSearch(raw_query="q=%23apple+%23iphone+%23iphonex&result_type=mixed&since=2017-09-10&count=100&lang=en")

forma = [(a.created_at, 
          a.text,
          a.retweet_count, 
          a.favorite_count,
          a.id, 
          a.user.name, 
          a.user.screen_name,
          a.user.description,
          a.user.id,
          a.user.location, 
          a.user.favourites_count, 
          a.user.followers_count,
          a.user.friends_count, 
          a.user.statuses_count, 
          a.user.created_at, 
          a.user.verified, 
          a.user.time_zone, 
          a.coordinates, 
          a.in_reply_to_status_id, 
          a.in_reply_to_user_id, 
          a.in_reply_to_screen_name) for a in results]
#print(forma)
#https://dev.twitter.com/overview/api/tweets#obj-coordinates

In [37]:
while tweetCount < 300:
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.GetSearch(raw_query="q=%23apple+%23iphone+%23iphonex&result_type=mixed&since=2017-09-10&count=100&lang=en")
            else:
                new_tweets = api.GetSearch(raw_query="q=%23apple+%23iphone+%23iphonex&result_type=mixed&since=2017-09-10&count=100&lang=en", 
                                      since_id=sinceId)
        else:
            if (not sinceId):
                new_tweets = api.GetSearch(raw_query="q=%23apple+%23iphone+%23iphonex&result_type=mixed&since=2017-09-10&count=100&lang=en",
                                        max_id=str(max_id - 1))
            else:
                new_tweets = api.GetSearch(raw_query="q=%23apple+%23iphone+%23iphonex&result_type=mixed&since=2017-09-10&count=100&lang=en",
                                        max_id=str(max_id - 1), since_id=sinceId)
        if not new_tweets:
            print("No more tweets found")
            break
        for tweet in new_tweets:
            print(tweet)
        tweetCount += len(new_tweets)
        print("Downloaded {0} tweets".format(tweetCount))
        max_id = new_tweets[-1].id
    except api.error.TwitterError as e:
        # Just exit if any error
        print("some error : " + str(e))
        break

{"created_at": "Wed Sep 13 13:01:19 +0000 2017", "favorite_count": 412, "hashtags": [{"text": "AppleEvent"}, {"text": "AppleEvent2017"}, {"text": "apple"}, {"text": "iPhone8"}, {"text": "iPhoneX"}], "id": 907952332746493952, "id_str": "907952332746493952", "lang": "en", "retweet_count": 223, "source": "<a href=\"http://twitter.com\" rel=\"nofollow\">Twitter Web Client</a>", "text": "I can't decide. What do u think??\n\nhttps://t.co/zSOEO7ZVvK\n\n#AppleEvent #AppleEvent2017 #apple #iPhone8 #iPhoneX\u2026 https://t.co/JgeUk16SRM", "truncated": true, "urls": [{"expanded_url": "http://bit.ly/2h1Prev", "url": "https://t.co/zSOEO7ZVvK"}, {"expanded_url": "https://twitter.com/i/web/status/907952332746493952", "url": "https://t.co/JgeUk16SRM"}], "user": {"created_at": "Mon Jun 10 16:17:47 +0000 2013", "description": "Described by industry insiders as an emotionally powerful vocal style masterfully united with a fusion of classic 80s rock & modern music! #newmusic #80smusic", "favourites_count"

{"created_at": "Wed Sep 13 13:01:19 +0000 2017", "favorite_count": 412, "hashtags": [{"text": "AppleEvent"}, {"text": "AppleEvent2017"}, {"text": "apple"}, {"text": "iPhone8"}, {"text": "iPhoneX"}], "id": 907952332746493952, "id_str": "907952332746493952", "lang": "en", "retweet_count": 223, "source": "<a href=\"http://twitter.com\" rel=\"nofollow\">Twitter Web Client</a>", "text": "I can't decide. What do u think??\n\nhttps://t.co/zSOEO7ZVvK\n\n#AppleEvent #AppleEvent2017 #apple #iPhone8 #iPhoneX\u2026 https://t.co/JgeUk16SRM", "truncated": true, "urls": [{"expanded_url": "http://bit.ly/2h1Prev", "url": "https://t.co/zSOEO7ZVvK"}, {"expanded_url": "https://twitter.com/i/web/status/907952332746493952", "url": "https://t.co/JgeUk16SRM"}], "user": {"created_at": "Mon Jun 10 16:17:47 +0000 2013", "description": "Described by industry insiders as an emotionally powerful vocal style masterfully united with a fusion of classic 80s rock & modern music! #newmusic #80smusic", "favourites_count"

{"created_at": "Wed Sep 13 13:01:19 +0000 2017", "favorite_count": 412, "hashtags": [{"text": "AppleEvent"}, {"text": "AppleEvent2017"}, {"text": "apple"}, {"text": "iPhone8"}, {"text": "iPhoneX"}], "id": 907952332746493952, "id_str": "907952332746493952", "lang": "en", "retweet_count": 223, "source": "<a href=\"http://twitter.com\" rel=\"nofollow\">Twitter Web Client</a>", "text": "I can't decide. What do u think??\n\nhttps://t.co/zSOEO7ZVvK\n\n#AppleEvent #AppleEvent2017 #apple #iPhone8 #iPhoneX\u2026 https://t.co/JgeUk16SRM", "truncated": true, "urls": [{"expanded_url": "http://bit.ly/2h1Prev", "url": "https://t.co/zSOEO7ZVvK"}, {"expanded_url": "https://twitter.com/i/web/status/907952332746493952", "url": "https://t.co/JgeUk16SRM"}], "user": {"created_at": "Mon Jun 10 16:17:47 +0000 2013", "description": "Described by industry insiders as an emotionally powerful vocal style masterfully united with a fusion of classic 80s rock & modern music! #newmusic #80smusic", "favourites_count"

In [38]:
#https://github.com/agalea91/nhl_2016_playoffs_twitter/blob/master/nhl_playoffs_2016_twitter_study.ipynb
data = {'created_at': [],'text': [], 
        'retweet_count': [], 'tweet_id':[],
        'user_favorites_count': [], 'tweet_location': [],
        'user_followers_count':[], 'user_time_zone':[],
        'user_id':[], 'user_name':[], 'in_reply_to_status_id':[], 'in_reply_to_user_id':[], 'in_reply_to_screen_name':[],
        "screen_name":[], 'user_verified':[],
        'user_location':[], 'user_description':[], 
        'user_statuses_count': [], 'user_created_at':[],
        'user_friends_count': [], 'favorite_count': []}

for t in forma:
    data['created_at'].append(t[0])
    data['text'].append(t[1])
    data['retweet_count'].append(t[2])
    data['favorite_count'].append(t[3])
    data['tweet_id'].append(t[4])
    data['user_name'].append(t[5])
    data['screen_name'].append(t[6])
    data['user_description'].append(t[7])
    data['user_id'].append(t[8])
    data['user_location'].append(t[9])
    data['user_favorites_count'].append(t[10])
    data['user_followers_count'].append(t[11])
    data['user_friends_count'].append(t[12])
    data['user_statuses_count'].append(t[13])
    data['user_created_at'].append(t[14])
    data['user_verified'].append(t[15])
    data['user_time_zone'].append(t[16])
    data['tweet_location'].append(t[17])
    data['in_reply_to_status_id'].append(t[18])
    data['in_reply_to_user_id'].append(t[19])
    data['in_reply_to_screen_name'].append(t[20])

df = pd.DataFrame(data)
df.drop_duplicates(inplace=True)
df['created_at'] = pd.to_datetime(df['created_at'])
df['user_created_at'] = pd.to_datetime(df['user_created_at'])

RT = []
for t in data['text']:
    RT.append(t.split()[0]=='RT')
df['RT'] = RT

df.head()

,created_at,favorite_count,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_count,screen_name,text,tweet_id,tweet_location,...,user_favorites_count,user_followers_count,user_friends_count,user_id,user_location,user_name,user_statuses_count,user_time_zone,user_verified,RT
0,2017-09-13 13:01:19,412,None,None,None,223,SarantosMelogia,I can't decide. What do u think??\n\nhttps://t...,907952332746493952,None,...,52694,903847,29652,1498656096,"Chicago, IL",Sarantos,10201,Central Time (US & Canada),True,False
1,2017-09-14 15:09:24,15,None,None,None,23,THINKgovuk,Here’s everything you need to know about the n...,908346952382861313,None,...,511,22243,448,1322025240,Great Britain,THINK! road safety,2292,Edinburgh,True,False
2,2017-09-18 07:30:05,21,None,None,None,30,THINKgovuk,Home button or no home button 😉 it's still dan...,909680911515602945,None,...,511,22243,448,1322025240,Great Britain,THINK! road safety,2292,Edinburgh,True,False
3,2017-09-21 11:28:39,0,None,None,None,0,GRIFFINDOOR88,IPHONE MAKERS HAVENT SEEN FACEOFF\n#griffindoo...,910828114321666048,None,...,904,807,1535,846015385,"SoCal, CA",GRIFFINDOOR88,7018,Pacific Time (US & Canada),False,False
4,2017-09-21 11:22:05,0,None,None,None,0,shortystretch13,Got my grubby lil mits on the new #iPhone X (-...,910826461858353152,None,...,1773,447,690,25485490,Coventry,Joey Short,5885,London,False,False


In [39]:
df.describe()

,favorite_count,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count
count,100.000000,100.000000,1.000000e+02,100.000000,100.000000,100.000000,1.000000e+02,100.000000
mean,4.980000,6.910000,9.105837e+17,8037.220000,11850.160000,1601.530000,2.179681e+17,26497.650000
std,41.256522,34.315852,3.761067e+14,53055.583703,90395.783764,4262.282851,3.713968e+17,84612.172372
min,0.000000,0.000000,9.079523e+17,0.000000,1.000000,2.000000,7.948150e+05,13.000000
25%,0.000000,0.000000,9.105544e+17,48.000000,155.250000,119.000000,1.534913e+08,820.750000
50%,0.000000,0.000000,9.106205e+17,397.000000,434.000000,422.000000,2.208677e+09,3736.500000
75%,0.000000,2.000000,9.107279e+17,2169.000000,2223.500000,1197.000000,7.102863e+17,15669.000000
max,412.000000,258.000000,9.108281e+17,529124.000000,903847.000000,29652.000000,9.101991e+17,735267.000000


In [40]:
len(np.unique(df.text))

93

In [41]:
df.columns[df.isnull().any()]

Index(['in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'tweet_location', 'user_time_zone'],
      dtype='object')

In [42]:
#https://cloud.google.com/natural-language/docs/sentiment-tutorial

In [43]:
engine = create_engine('mysql+pymysql://root:@localhost/sawi_tweets?charset=utf8mb4', encoding='utf8', echo = True)
df.to_sql('sawi_tweets_historical', engine, if_exists='replace')

2017-09-21 13:55:32,101 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2017-09-21 13:55:32,107 INFO sqlalchemy.engine.base.Engine {}
2017-09-21 13:55:32,114 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2017-09-21 13:55:32,121 INFO sqlalchemy.engine.base.Engine {}
2017-09-21 13:55:32,129 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
2017-09-21 13:55:32,136 INFO sqlalchemy.engine.base.Engine {}
2017-09-21 13:55:32,142 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2017-09-21 13:55:32,148 INFO sqlalchemy.engine.base.Engine {}
2017-09-21 13:55:32,153 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2017-09-21 13:55:32,159 INFO sqlalchemy.engine.base.Engine {}
2017-09-21 13:55:32,165 INFO sqlalchemy.engine.base.Engine SELECT CAST('test collated returns' AS CHAR CHARACTER SET utf8) COLLATE utf8_bin AS anon_1
2017-09-21 13

/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_1` CHECK (`user_verified` in (0,1)),'
  util.warn("Unknown schema content: %r" % line)
/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_2` CHECK (`RT` in (0,1))'
  util.warn("Unknown schema content: %r" % line)


2017-09-21 13:55:32,468 INFO sqlalchemy.engine.base.Engine COMMIT
